In [1]:
import sys

print(sys.executable)
sys.path.append('/Users/xu081/Documents/trec_t2')
for p in sys.path:
    print(p)

/Users/xu081/opt/anaconda3/bin/python
/Users/xu081/Documents/trec_t2/notebooks
/Users/xu081/opt/anaconda3/lib/python37.zip
/Users/xu081/opt/anaconda3/lib/python3.7
/Users/xu081/opt/anaconda3/lib/python3.7/lib-dynload

/Users/xu081/opt/anaconda3/lib/python3.7/site-packages
/Users/xu081/opt/anaconda3/lib/python3.7/site-packages/aeosa
/Users/xu081/Documents/trec_t2/local_parser
/Users/xu081/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions
/Users/xu081/.ipython
/Users/xu081/Documents/trec_t2


In [2]:
!which python3
!echo ${PATH}

/Users/xu081/opt/anaconda3/bin/python3
/Users/xu081/opt/anaconda3/bin:/Users/xu081/opt/anaconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/cai065/.local/bin:/Library/TeX/texbin:/usr/local/laps:/usr/local/munki


In [3]:
import os
import pandas as pd

from collections import defaultdict

from typing import Any, List, Dict, Tuple, Sequence

from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh.qparser import QueryParser, MultifieldParser, OrGroup
from whoosh import scoring

In [4]:
import local_parser
from local_parser.extract_trial_data import store_ids
from local_parser.build_bert_ready_dataset import parse_topics, final_process_df

In [5]:
os.path.dirname(local_parser.__file__)

'/Users/xu081/Documents/trec_t2/local_parser'

In [6]:
pre = pd.read_pickle(
    "../A2A4UMA/pre_rerank_data_files/result_sets_pre.pickle"
)
post = pd.read_pickle(
    "../A2A4UMA/pre_rerank_data_files/results_post.pickle"
)

full_df = pd.read_pickle(
    "../data/trials_topics_combined_all_years.pickle"
)

In [7]:
pre[0]["ranking"]["1"][:3]

[['NCT01209598', 51.3797624360671, 'All', '', 6570],
 ['NCT03096912', 49.04213434180528, 'All', '', 6570],
 ['NCT02571829', 48.29976885555331, 'All', '', 6570]]

In [8]:
post[0]["ranking"]["1"][:3]

[['NCT01209598', 51.3797624360671, 'All', '', 6570],
 ['NCT03096912', 49.04213434180528, 'All', '', 6570],
 ['NCT02571829', 48.29976885555331, 'All', '', 6570]]

## Retrieving doc_ids for Re-ranking

In [9]:
pre[0]["ranking"]['3']

[['NCT02282917', 43.15832569891124, 'All', '', 6570],
 ['NCT02523014', 42.52122514555188, 'All', '', 6570],
 ['NCT00973739', 42.469934996826325, 'All', 29200, 1460],
 ['NCT02934256', 42.217021655556884, 'All', 18250, 5840],
 ['NCT02104323', 41.122347677659086, 'All', 10950, 5840],
 ['NCT00003483', 41.029994975574596, 'All', 36135, 6570],
 ['NCT00598351', 40.32110994802228, 'All', 27375, 2920],
 ['NCT03015701', 40.086631669735134, 'All', 43800, 6570],
 ['NCT03016091', 39.636337542183085, 'All', '', 6570],
 ['NCT00003590', 39.53941666017546, 'All', 43800, 6570],
 ['NCT02648997', 39.27261831909357, 'All', '', 6570],
 ['NCT00859040', 39.079858190169176, 'All', '', 6570],
 ['NCT01222728', 38.17618791430081, 'All', 18250, 6570],
 ['NCT00972335', 37.30471470606359, 'All', '', 6570],
 ['NCT01892397', 36.561784166374274, 'All', '', 6570],
 ['NCT00813592', 36.21661521890005, 'All', '', 6570],
 ['NCT02234050', 35.92490273658187, 'All', '', 6570],
 ['NCT00006119', 35.40554583420377, 'All', '', 584

In [10]:
def get_doc_ids(ranking_dict: list, top_n: int) -> Dict[str, List]:
    topic_doc_id_dict = defaultdict(list)
    
    for key in ranking_dict[RANKING].keys():
        idx = 0
        while idx < top_n:
            topic_doc_id_dict[key].append(ranking_dict[RANKING][key][idx][0])
            idx += 1
    
    return topic_doc_id_dict

def check_missing_ids(
    topic_doct_id_dict: Dict[str, List], 
    trials_df: pd.DataFrame) -> List[str]:
    """
    
    Check for missing IDs one-by-one. This is very useful if 
    the labelled datasets is missing something. 
    
    Now, we don't need the labels to do the re-ranking as we will be
    outputting a regression.
    
    """
    missing_doc_id = []
    missing_topic = []
    for key in topic_doct_id_dict.keys():
        for doc_id in topic_doct_id_dict[key]:
            if trials_df[trials_df["id"] == doc_id].shape[0] == 0:
                print(f"{doc_id} not found in DataFrame!")
                missing_doc_id.append(doc_id)
                missing_topic.append(key)
    return missing_doc_id, missing_topic

def get_missing_ids(
    missing_ids: List[str],
    index: str, qf: str,
    return_fields: List[str], 
    task: str, 
    num_procs: int,
    parallel: bool) -> Dict[str, Any]:
    
    res = store_ids(
        t_ids=missing_ids,
        index=index, qf=qf,
        return_fields=return_fields, task=task,
        bm25f_params={}, num_procs=num_procs,
        parallel=parallel
    )
    
    return res
    
def filter_df():
    pass

In [11]:
def find_idx(ranking_dict: list, topic: str, doc_id: str) -> int:
    """
    
    Returns index position of the array that holds the relevant doc_id
    
    """
    idx = 0
    for x in ranking_dict[0]["ranking"][topic]:
        if x[0] == doc_id:
            return idx
        idx += 1

def update_score(topic: str, year: int, doc_id: str, score: float) -> None:
    pass

In [12]:
RANKING = "ranking"

In [13]:
t_d_dict = get_doc_ids(pre[0], 50)
t_d_dict['1'][:5]

['NCT01209598', 'NCT03096912', 'NCT02571829', 'NCT03034252', 'NCT02343172']

In [14]:
missing_ids, missing_topics = check_missing_ids(t_d_dict, full_df)
print(missing_ids[:2])
print(missing_topics[:2])

NCT01857661 not found in DataFrame!
NCT00949312 not found in DataFrame!
NCT02493569 not found in DataFrame!
NCT00496288 not found in DataFrame!
NCT01879189 not found in DataFrame!
NCT02550210 not found in DataFrame!
NCT00897962 not found in DataFrame!
NCT00473096 not found in DataFrame!
NCT02662868 not found in DataFrame!
NCT00584142 not found in DataFrame!
NCT00416975 not found in DataFrame!
NCT00148720 not found in DataFrame!
NCT01181518 not found in DataFrame!
NCT02890095 not found in DataFrame!
NCT00676793 not found in DataFrame!
NCT00148057 not found in DataFrame!
NCT00347568 not found in DataFrame!
NCT01368744 not found in DataFrame!
NCT00078832 not found in DataFrame!
NCT01266642 not found in DataFrame!
['NCT01857661', 'NCT00949312']
['1', '2']


In [15]:
missing_dict = dict(zip(missing_ids, missing_topics))
missing_dict

{'NCT01857661': '1',
 'NCT00949312': '2',
 'NCT02493569': '4',
 'NCT00496288': '4',
 'NCT01879189': '4',
 'NCT02550210': '4',
 'NCT00897962': '4',
 'NCT00473096': '4',
 'NCT02662868': '4',
 'NCT00584142': '4',
 'NCT00416975': '4',
 'NCT00148720': '4',
 'NCT01181518': '4',
 'NCT02890095': '4',
 'NCT00676793': '4',
 'NCT00148057': '4',
 'NCT00347568': '4',
 'NCT01368744': '4',
 'NCT00078832': '4',
 'NCT01266642': '4'}

In [16]:
qf = "id^1"
bm25f_params={}
year = "2017"
task = "clinical"
num_procs = 1
parallel = False
    
ct_17_path = "../A2A4UMA/indices/ct17_whoosh"
ct_17_idx = open_dir(ct_17_path)

return_fields = [
    "score",
    "id", "brief_summary", "brief_title",
    "minimum_age", "gender",
    "primary_outcome", "detailed_description",
    "keywords", "official_title",
    "intervention_type",
    "intervention_name",
    "intervention_browse",
    "condition_browse",
    "inclusion", "exclusion",
]

In [17]:
res = get_missing_ids(
    missing_ids=missing_ids,
    index=ct_17_idx, qf=qf, 
    return_fields=return_fields,
    task=task, num_procs=num_procs,
    parallel=parallel
)

Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Default scoring
Time elapsed: 37.992295026779175


In [18]:
res

[([{'score': 1.0,
    'id': 'NCT01857661',
    'brief_summary': 'The University of Sao Paulo Department of Otorhinolaryngology developed prototypes for a digital new hearing aid with an integrated sound generator. These prototypes assist both hearing rehabilitation (about 10% of the population) and tinnitus sufferers (about 17% of the population). Currently, the Brazilian Public Heath Care does not have any such hearing aids. So, the development of devices for the hearing rehabilitation as well as the treatment of tinnitus represents an advance in the implementation of Public Policies in Brazil. The aim of this study is to compare the efficacy of the hearing aid with an integrated sound generator and amplification alone for tinnitus control in patients with tinnitus associated hearing loss.',
    'brief_title': 'The Influence of the Sound Generator Combined With Conventional Amplification for Tinnitus Control: Blind Randomized Clinical Trial',
    'minimum_age': 6570,
    'gender': 'Al

In [19]:
len(res)

20

In [20]:
res[4][0][0]["score"]

1.0

In [21]:
df_arr = []

for i in range(len(res)):
    df_arr_tmp = []
    df_arr_tmp += res[i][0]

    print(f"i, len(df[i][j]): {i}, {len(res[i])}")
    df_arr += df_arr_tmp

i, len(df[i][j]): 0, 2
i, len(df[i][j]): 1, 2
i, len(df[i][j]): 2, 2
i, len(df[i][j]): 3, 2
i, len(df[i][j]): 4, 2
i, len(df[i][j]): 5, 2
i, len(df[i][j]): 6, 2
i, len(df[i][j]): 7, 2
i, len(df[i][j]): 8, 2
i, len(df[i][j]): 9, 2
i, len(df[i][j]): 10, 2
i, len(df[i][j]): 11, 2
i, len(df[i][j]): 12, 2
i, len(df[i][j]): 13, 2
i, len(df[i][j]): 14, 2
i, len(df[i][j]): 15, 2
i, len(df[i][j]): 16, 2
i, len(df[i][j]): 17, 2
i, len(df[i][j]): 18, 2
i, len(df[i][j]): 19, 2


In [22]:
for doc_id, topic_num in list(zip(missing_ids, missing_topics)):
    print(find_idx(pre, topic_num, doc_id))

41
21
2
5
7
14
16
17
19
20
22
23
24
29
31
37
38
44
48
49


In [23]:
print(list(zip(missing_ids, missing_topics)))

[('NCT01857661', '1'), ('NCT00949312', '2'), ('NCT02493569', '4'), ('NCT00496288', '4'), ('NCT01879189', '4'), ('NCT02550210', '4'), ('NCT00897962', '4'), ('NCT00473096', '4'), ('NCT02662868', '4'), ('NCT00584142', '4'), ('NCT00416975', '4'), ('NCT00148720', '4'), ('NCT01181518', '4'), ('NCT02890095', '4'), ('NCT00676793', '4'), ('NCT00148057', '4'), ('NCT00347568', '4'), ('NCT01368744', '4'), ('NCT00078832', '4'), ('NCT01266642', '4')]


In [24]:
df_missing = pd.DataFrame(df_arr)
df_missing["topic"] = missing_topics
df_missing.iloc[:2]

,score,id,brief_summary,brief_title,minimum_age,gender,primary_outcome,detailed_description,keywords,official_title,intervention_type,intervention_name,intervention_browse,condition_browse,inclusion,exclusion,topic
0,1.0,NCT01857661,The University of Sao Paulo Department of Otor...,The Influence of the Sound Generator Combined ...,6570,All,THI 3 months tinnitus handicap inventory,"This study, in the form of a blind randomized ...",tinnitus hearing loss hearing devices,The Influence of the Sound Generator Combined ...,Device Device,sound generator hearing aids with only amplifi...,,Hearing Loss Deafness Tinnitus,- mild to moderate sensorineural hearing loss ...,- profound hearing loss - conductive hearing l...,1
1,1.0,NCT00949312,RATIONALE: Diagnostic procedures that look for...,Studying Lymph Nodes in Patients With Stage II...,6570,All,Disease-free survival (time to local or distan...,OBJECTIVES: - Determine whether the immunohist...,stage II colon cancer,Ultrastaging of Early Colon Cancer,Genetic Genetic Genetic Other Procedure Other,RNA marker analysis of lymph node and primary ...,,Colorectal Neoplasms Colonic Neoplasms,All of the following inclusion criteria must b...,Any of the following criteria will exclude the...,2


__Change topic to int__ to match with XML files

In [25]:
df_missing["topic"] = df_missing["topic"].astype(int)

In [26]:
year = 2017

TOPICS_PATH = f"../data/pm_labels_{year}"
TOPICS_XML = f"topics{year}.xml"

topics_dict = parse_topics(TOPICS_PATH, TOPICS_XML)

##### COMBINING TOPICS AND CLINICAL TRIALS #####

df_missing["topic_info"] = df_missing["topic"].map(topics_dict)
df_missing.head()

child.attrib["number"]: 10
['Lung adenocarcinoma', 'KRAS (G12C)', '61-year-old female', 'Hypertension, Hypercholesterolemia']
child.attrib["number"]: 20
['Liposarcoma', 'MDM2 Amplification', '26-year-old male', 'None']
child.attrib["number"]: 30
['Pancreatic adenocarcinoma', 'RB1, TP53, KRAS', '57-year-old female', 'None']


,score,id,brief_summary,brief_title,minimum_age,gender,primary_outcome,detailed_description,keywords,official_title,intervention_type,intervention_name,intervention_browse,condition_browse,inclusion,exclusion,topic,topic_info
0,1.0,NCT01857661,The University of Sao Paulo Department of Otor...,The Influence of the Sound Generator Combined ...,6570,All,THI 3 months tinnitus handicap inventory,"This study, in the form of a blind randomized ...",tinnitus hearing loss hearing devices,The Influence of the Sound Generator Combined ...,Device Device,sound generator hearing aids with only amplifi...,,Hearing Loss Deafness Tinnitus,- mild to moderate sensorineural hearing loss ...,- profound hearing loss - conductive hearing l...,1,"[Liposarcoma, CDK4 Amplification, 38-year-old ..."
1,1.0,NCT00949312,RATIONALE: Diagnostic procedures that look for...,Studying Lymph Nodes in Patients With Stage II...,6570,All,Disease-free survival (time to local or distan...,OBJECTIVES: - Determine whether the immunohist...,stage II colon cancer,Ultrastaging of Early Colon Cancer,Genetic Genetic Genetic Other Procedure Other,RNA marker analysis of lymph node and primary ...,,Colorectal Neoplasms Colonic Neoplasms,All of the following inclusion criteria must b...,Any of the following criteria will exclude the...,2,"[Colon cancer, KRAS (G13D), BRAF (V600E), 52-y..."
2,1.0,NCT02493569,The purpose of the study is to study the epide...,A Multicenter Study of Clinical Epidemiology o...,,Female,Overall five-year survival for the female brea...,"The study is a multicentric, retrospective cli...","Breast cancer, hospital, epidemiology, diagnos...",A Multicenter Study of Clinical Epidemiology o...,Procedure,The radical mastectomy of breast cancer,,Breast Neoplasms,- Female patients; - The hospitalized patients...,- Male breast cancer patients; - Patients with...,4,"[Breast cancer, FGFR1 Amplification, PTEN (Q17..."
3,1.0,NCT00496288,Women with BRCA germline mutations face a very...,Prophylactic Irradiation to the Contralateral ...,10950,Female,Rate of contralateral breast cancer in carrier...,Trial design and statistics: This is a multi c...,BRCA mutation Breast cancer Prophylactic breas...,Phase II Multicenter Clinical Trial of Prophyl...,Radiation,Prophylactic radiation to the contralateral br...,,Breast Neoplasms,1. Female patient diagnosed with stage I-III b...,1. Metastatic breast cancer. 2. Previous irrad...,4,"[Breast cancer, FGFR1 Amplification, PTEN (Q17..."
4,1.0,NCT01879189,In order to better communicate personal risk o...,PROSPR/PCIPS Project 3: Communication of Perso...,14235,Female,Mammography Use Correlated to Personal Risk of...,In order to better communicate personal risk o...,,PROSPR/PCIPS Project 3: Communication of Perso...,Behavioral,Breast Cancer Screening Decision Aid,,Breast Neoplasms,- Women ages 39-48 - Patient of one of the nin...,- Prior diagnosis of breast cancer - Major com...,4,"[Breast cancer, FGFR1 Amplification, PTEN (Q17..."


In [27]:
year = 2017

df_missing = final_process_df(df_missing, year=year)

print(df_missing.shape)
df_missing.head()

(20, 21)


,score,id,brief_summary,brief_title,minimum_age,gender,primary_outcome,detailed_description,keywords,official_title,...,intervention_name,intervention_browse,condition_browse,inclusion,exclusion,topic,disease,gene,age,year
0,1.0,NCT01857661,The University of Sao Paulo Department of Otor...,The Influence of the Sound Generator Combined ...,6570,male,THI 3 months tinnitus handicap inventory,"This study, in the form of a blind randomized ...",tinnitus hearing loss hearing devices,The Influence of the Sound Generator Combined ...,...,sound generator hearing aids with only amplifi...,,Hearing Loss Deafness Tinnitus,- mild to moderate sensorineural hearing loss ...,- profound hearing loss - conductive hearing l...,1,Liposarcoma,CDK4 Amplification,None,2017
1,1.0,NCT00949312,RATIONALE: Diagnostic procedures that look for...,Studying Lymph Nodes in Patients With Stage II...,6570,male,Disease-free survival (time to local or distan...,OBJECTIVES: - Determine whether the immunohist...,stage II colon cancer,Ultrastaging of Early Colon Cancer,...,RNA marker analysis of lymph node and primary ...,,Colorectal Neoplasms Colonic Neoplasms,All of the following inclusion criteria must b...,Any of the following criteria will exclude the...,2,Colon cancer,"KRAS (G13D), BRAF (V600E)",None,2017
2,1.0,NCT02493569,The purpose of the study is to study the epide...,A Multicenter Study of Clinical Epidemiology o...,,female,Overall five-year survival for the female brea...,"The study is a multicentric, retrospective cli...","Breast cancer, hospital, epidemiology, diagnos...",A Multicenter Study of Clinical Epidemiology o...,...,The radical mastectomy of breast cancer,,Breast Neoplasms,- Female patients; - The hospitalized patients...,- Male breast cancer patients; - Patients with...,4,Breast cancer,"FGFR1 Amplification, PTEN (Q171)",None,2017
3,1.0,NCT00496288,Women with BRCA germline mutations face a very...,Prophylactic Irradiation to the Contralateral ...,10950,female,Rate of contralateral breast cancer in carrier...,Trial design and statistics: This is a multi c...,BRCA mutation Breast cancer Prophylactic breas...,Phase II Multicenter Clinical Trial of Prophyl...,...,Prophylactic radiation to the contralateral br...,,Breast Neoplasms,1. Female patient diagnosed with stage I-III b...,1. Metastatic breast cancer. 2. Previous irrad...,4,Breast cancer,"FGFR1 Amplification, PTEN (Q171)",None,2017
4,1.0,NCT01879189,In order to better communicate personal risk o...,PROSPR/PCIPS Project 3: Communication of Perso...,14235,female,Mammography Use Correlated to Personal Risk of...,In order to better communicate personal risk o...,,PROSPR/PCIPS Project 3: Communication of Perso...,...,Breast Cancer Screening Decision Aid,,Breast Neoplasms,- Women ages 39-48 - Patient of one of the nin...,- Prior diagnosis of breast cancer - Major com...,4,Breast cancer,"FGFR1 Amplification, PTEN (Q171)",None,2017


In [28]:
all_df = []

for key in t_d_dict.keys():
    tmp_df = pd.DataFrame()
    topic_arr = [key] * len(t_d_dict[key])
    
    tmp_df = pd.DataFrame(
        zip(topic_arr, t_d_dict[key]),
        columns=["topic", "id"]
    )
    all_df.append(tmp_df)
    
all_df = pd.concat(all_df)
print(all_df.shape)
print(all_df["topic"].unique())
print(all_df.dtypes)
all_df.head()

(250, 2)
['1' '2' '3' '4' '5']
topic    object
id       object
dtype: object


,topic,id
0,1,NCT01209598
1,1,NCT03096912
2,1,NCT02571829
3,1,NCT03034252
4,1,NCT02343172


In [29]:
SUBSET_COLUMNS = [
    "id", 
    "brief_summary", 
    "brief_title", 
    "disease", "gene"
]

df_concat = pd.concat(
    [
        full_df[SUBSET_COLUMNS], 
        df_missing[SUBSET_COLUMNS]
    ],
    axis=0,
    join="inner"
)
print(df_concat.shape)
df_concat.head()

(40223, 5)


,id,brief_summary,brief_title,disease,gene
0,NCT01774162,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,Pancreatic cancer,CDK6 Amplification
1,NCT01774162,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,Pancreatic adenocarcinoma,"KRAS, TP53"
2,NCT01774162,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,Pancreatic ductal adenocarcinoma,ERBB3
3,NCT01774162,Endoscopic ultrasound (EUS) is a well-establis...,EUS-guided Fine Needle Biopsy With a New Core ...,Pancreatic adenocarcinoma,"RB1, TP53, KRAS"
4,NCT01226147,An open-label study to evaluate the efficacy a...,Efficacy and Safety of Tamibarotene(AM80) for ...,Lung cancer,EGFR (L858R)


In [30]:
SUBSET_COLUMNS = [
    "id", 
    "brief_summary", 
    "brief_title", 
    "disease", "gene"
]

all_df["topic"] = all_df["topic"].astype(int)
full_df["topic"] = full_df["topic"].astype(int)
all_df["id"] = all_df["id"].astype(str)
full_df["id"] = full_df["id"].astype(str)


all_df_merge = all_df.merge(df_concat, on="id", how="left")
all_df_merge.head()

,topic,id,brief_summary,brief_title,disease,gene
0,1,NCT01209598,The purpose of this study is to find out what ...,PD0332991 (Palbociclib) in Patients With Advan...,Liposarcoma,CDK4 Amplification
1,1,NCT01209598,The purpose of this study is to find out what ...,PD0332991 (Palbociclib) in Patients With Advan...,Pancreatic cancer,CDK6 Amplification
2,1,NCT01209598,The purpose of this study is to find out what ...,PD0332991 (Palbociclib) in Patients With Advan...,Liposarcoma,MDM2 Amplification
3,1,NCT01209598,The purpose of this study is to find out what ...,PD0332991 (Palbociclib) in Patients With Advan...,sarcoma,MDM2
4,1,NCT01209598,The purpose of this study is to find out what ...,PD0332991 (Palbociclib) in Patients With Advan...,gastrointestinal stromal tumor,KIT (V654A)


In [31]:
for col in all_df_merge.columns:
    print(all_df_merge[col].isna().value_counts())

False    1751
Name: topic, dtype: int64
False    1751
Name: id, dtype: int64
False    1751
Name: brief_summary, dtype: int64
False    1751
Name: brief_title, dtype: int64
False    1751
Name: disease, dtype: int64
False    1751
Name: gene, dtype: int64


In [32]:
print(all_df_merge[all_df_merge["brief_title"].isna() == True].shape)
all_df_merge[all_df_merge["brief_title"].isna() == True]

(0, 6)


,topic,id,brief_summary,brief_title,disease,gene


In [33]:
df_missing = df_missing[SUBSET_COLUMNS]
print(df_missing.shape)
df_missing.head()

(20, 5)


,id,brief_summary,brief_title,disease,gene
0,NCT01857661,The University of Sao Paulo Department of Otor...,The Influence of the Sound Generator Combined ...,Liposarcoma,CDK4 Amplification
1,NCT00949312,RATIONALE: Diagnostic procedures that look for...,Studying Lymph Nodes in Patients With Stage II...,Colon cancer,"KRAS (G13D), BRAF (V600E)"
2,NCT02493569,The purpose of the study is to study the epide...,A Multicenter Study of Clinical Epidemiology o...,Breast cancer,"FGFR1 Amplification, PTEN (Q171)"
3,NCT00496288,Women with BRCA germline mutations face a very...,Prophylactic Irradiation to the Contralateral ...,Breast cancer,"FGFR1 Amplification, PTEN (Q171)"
4,NCT01879189,In order to better communicate personal risk o...,PROSPR/PCIPS Project 3: Communication of Perso...,Breast cancer,"FGFR1 Amplification, PTEN (Q171)"


In [34]:
for col in all_df_merge.columns:
    print(all_df_merge[col].isna().value_counts())

False    1751
Name: topic, dtype: int64
False    1751
Name: id, dtype: int64
False    1751
Name: brief_summary, dtype: int64
False    1751
Name: brief_title, dtype: int64
False    1751
Name: disease, dtype: int64
False    1751
Name: gene, dtype: int64


## Updating the Score after Re-ranking

In [35]:
find_id(post, "1", "NCT01777776")

NameError: name 'find_id' is not defined